In [1]:
import sys
sys.executable

'/home/ccrs70/miniconda3/bin/python'

In [ ]:
from pathlib import Path

In [ ]:
# Updated imports to support OCR configuration
from docling.datamodel.document import InputFormat
from docling.document_converter import DocumentConverter, PdfFormatOption
from docling.datamodel.base_models import ConversionStatus
from docling.datamodel.pipeline_options import PdfPipelineOptions

In [ ]:
notebook_dir = Path.cwd()
PROJECT_ROOT = notebook_dir.parent
filename = "ch1_ch14_Brain_and_behavior"

In [ ]:
print(f"Notebook is in: {notebook_dir}")
print(f"Project root is: {PROJECT_ROOT}")

In [ ]:
# Construct the full path and then add the .pdf suffix
source_file = (PROJECT_ROOT / "data" / "raw" / "neuroscience" / filename).with_suffix(".pdf")

# Construct the full path and then add the .md suffix
destination_file = (PROJECT_ROOT / "data" / "processed" / "neuroscience" / filename).with_suffix(".md")



## conversion no_ocr no_tables

In [ ]:
# 1. Define Pipeline Options
pipeline_options = PdfPipelineOptions()
pipeline_options.do_ocr = False           # <--- Force OCR OFF (Fast & clean for digital docs)
pipeline_options.do_table_structure = False # Keep table recognition ON

In [ ]:
# 2. Initialize Converter with specific PDF options
converter = DocumentConverter(
    format_options={
        InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)
    }
)

In [ ]:
result = converter.convert(source_file)

In [ ]:
# Access the internal DoclingDocument object
doc = result.document

## save markdown and json 

In [ ]:
destination_file.write_text(result.document.export_to_markdown(), encoding="utf-8")

In [ ]:
destination_json = (PROJECT_ROOT / "data" / "processed" / "neuroscience" / filename).with_suffix(".json")

In [ ]:
# 4. Option B: Save to a JSON file for easier inspection in a text editor
doc.save_as_json(destination_json)

## possible labels of docling items
https://github.com/docling-project/docling-core/blob/main/docling_core/types/doc/labels.py
```
class DocItemLabel(str, Enum):
    """DocItemLabel."""

    CAPTION = "caption"
    CHART = "chart"
    FOOTNOTE = "footnote"
    FORMULA = "formula"
    LIST_ITEM = "list_item"
    PAGE_FOOTER = "page_footer"
    PAGE_HEADER = "page_header"
    PICTURE = "picture"
    SECTION_HEADER = "section_header"
    TABLE = "table"
    TEXT = "text"
    TITLE = "title"
    DOCUMENT_INDEX = "document_index"
    CODE = "code"
    CHECKBOX_SELECTED = "checkbox_selected"
    CHECKBOX_UNSELECTED = "checkbox_unselected"
    FORM = "form"
    KEY_VALUE_REGION = "key_value_region"
    GRADING_SCALE = "grading_scale"  # for elements in forms, questionaires representing a grading scale
    # e.g. [strongly disagree | ... | ... | strongly agree]
    # e.g. ★★☆☆☆
    HANDWRITTEN_TEXT = "handwritten_text"
    EMPTY_VALUE = "empty_value"  # used for empty value fields in fillable forms

    # Additional labels for markup-based formats (e.g. HTML, Word)
    PARAGRAPH = "paragraph"
    REFERENCE = "reference"```

## delete unwanted items

DocItemLabel.CAPTION, DocItemLabel.FOOTNOTE, 
                    DocItemLabel.PAGE_FOOTER, DocItemLabel.PAGE_HEADER, 
                    DocItemLabel.TABLE
                    

In [ ]:
from docling.document_converter import DocumentConverter
from docling.datamodel.document import DocItemLabel

In [ ]:
# 2. Identify items to remove (e.g., remove all Captions and Footnotes)
# We collect them in a list first to avoid modifying the tree while iterating
items_to_remove = []
labels_to_remove = {DocItemLabel.CAPTION, DocItemLabel.FOOTNOTE, 
                    DocItemLabel.PAGE_FOOTER, DocItemLabel.PAGE_HEADER, 
                    DocItemLabel.TABLE}

for item, level in doc.iterate_items():
    # Check if the item has a label and if it matches our target list
    if hasattr(item, "label") and item.label in labels_to_remove:
        items_to_remove.append(item)

# 3. Delete the items from the document
# This updates the document tree in-place
doc.delete_items(node_items=items_to_remove)

In [ ]:
filename_removeditems = filename + "_removeditems"
destination_file_removeditems = (PROJECT_ROOT / "data" / "processed" / "neuroscience" / filename_removeditems).with_suffix(".md")

In [ ]:
# 4. Now 'doc' can be passed to your RAG pipeline (e.g., chunking or export)
destination_file_removeditems.write_text(doc.export_to_markdown(), encoding="utf-8")

In [ ]:
destination_file_json_removeditems = (PROJECT_ROOT / "data" / "processed" / "neuroscience" / filename_removeditems).with_suffix(".json")

In [ ]:
doc.save_as_json(destination_file_json_removeditems)

## delete pictures and its children
removes picture elements that contain texts inside the picture. No OCR is done but small texts like (a) (b) digital text inside the picture makes the converter to create picture items (empty) that contains small texts. This is to remove that noise

In [ ]:
# 1. Helper function to find all children recursively
def get_all_descendants(item):
    """Recursively collect all children, grandchildren, etc."""
    descendants = []
    # Check if the item has children
    if hasattr(item, "children") and item.children:
        for child in item.children:
            descendants.append(child)
            # Recursively get children of the child
            descendants.extend(get_all_descendants(child))
    return descendants

In [ ]:
# 3. Identify items to remove
items_to_remove = []   # Use a list instead of a set
seen_ids = set()       # Track IDs to avoid duplicates

for item, level in doc.iterate_items():
    # Check if it is a Picture
    if hasattr(item, "label") and item.label == DocItemLabel.PICTURE:
        
        # A. Add the Picture item itself (if not already added)
        if id(item) not in seen_ids:
            items_to_remove.append(item)
            seen_ids.add(id(item))
        
        # B. Get all children (captions, texts inside)
        children = get_all_descendants(item)
        
        for child in children:
            if id(child) not in seen_ids:
                items_to_remove.append(child)
                seen_ids.add(id(child))

# 4. Delete the items
if items_to_remove:
    print(f"Removing {len(items_to_remove)} items...")
    doc.delete_items(node_items=items_to_remove)

In [ ]:
filename_removedpictures = filename + "_removedpictures"
destination_file_json_removedpictures= (PROJECT_ROOT / "data" / "processed" / "neuroscience" / filename_removedpictures).with_suffix(".json")

In [ ]:
doc.save_as_json(destination_file_json_removedpictures)

In [ ]:
destination_file_removedpictures= (PROJECT_ROOT / "data" / "processed" / "neuroscience" / filename_removedpictures).with_suffix(".md")

In [ ]:
destination_file_removedpictures.write_text(doc.export_to_markdown(), encoding="utf-8")

## scispacy to sementation, cleaning and named entity recognition

In [ ]:
import spacy
import re
from dataclasses import dataclass
from typing import List, Dict, Any

In [ ]:
# --- 1. DATA STRUCTURES ---
@dataclass
class ProcessedSentence:
    """
    Standardizes the output for every sentence.
    This is what we will pass to the Chunking Phase.
    """
    id: int                         # Sequence number (useful for ordering)
    text: str                       # The clean sentence content
    entities: List[str]             # List of extracted entities (e.g., 'Dopamine')
    entity_labels: List[str]        # List of entity types (e.g., 'SIMPLE_CHEMICAL')
    source_metadata: Dict[str, Any] # Grounding info (Book Title, Author)

In [ ]:
# --- 2. THE PROCESSOR CLASS ---
class NeuroProcessor:
    def __init__(self):
        print("⏳ Loading SciSpacy models...")
        
        # MODEL 1: Structural/Linguistic Model (en_core_sci_md)
        # Why: This model is trained on biomedical text, so it knows that "Fig. 1.2"
        # is not the end of a sentence. We use this ONLY for splitting sentences.
        try:
            self.nlp_seg = spacy.load("en_core_sci_md")
            # We disable NER here to save compute, we only want the parser for sentences
            self.nlp_seg.disable_pipes(["ner"]) 
        except OSError:
            raise OSError("Error: Could not load 'en_core_sci_md'. Please install it.")

        # MODEL 2: Specialized NER Model (en_ner_bionlp13cg_md)
        # Why: This provides specific granular tags like ORGAN, CELL, TISSUE, 
        # rather than the generic tags found in the core model.
        try:
            self.nlp_ner = spacy.load("en_ner_bionlp13cg_md")
        except OSError:
            raise OSError("Error: Could not load 'en_ner_bionlp13cg_md'. Please install it.")
            
        print("✅ Models loaded. Ready to process.")

    def process_text(self, markdown_text: str, metadata: Dict[str, Any]) -> List[ProcessedSentence]:
        """
        Input: Clean Markdown string + Book Metadata.
        Output: List of ProcessedSentence objects.
        """
        # A. Cleanup: Collapse multiple spaces/newlines into single spaces
        # This prevents the sentence splitter from getting confused by PDF line breaks
        clean_text = " ".join(markdown_text.split())
        
        # B. Segmentation (Model 1)
        doc = self.nlp_seg(clean_text)
        
        # Filter out extremely short artifacts (like page numbers "14", or headers "##")
        raw_sentences = [sent.text.strip() for sent in doc.sents if len(sent.text.strip()) > 10]
        
        processed_output = []

        # C. Enrichment (Model 2)
        print(f"   -> Processing {len(raw_sentences)} sentences for entities...")
        
        for i, sent_text in enumerate(raw_sentences):
            # Run NER on the individual sentence
            ner_doc = self.nlp_ner(sent_text)
            
            # Extract distinct entities and labels
            # We use set() to remove duplicates (e.g., if 'neuron' appears twice in one sentence)
            entities = list(set([ent.text for ent in ner_doc.ents]))
            entity_labels = list(set([ent.label_ for ent in ner_doc.ents]))
            
            # Create the object
            p_sent = ProcessedSentence(
                id=i,
                text=sent_text,
                entities=entities,
                entity_labels=entity_labels,
                source_metadata=metadata
            )
            processed_output.append(p_sent)
            
        return processed_output

In [ ]:
# --- 3. HELPER FOR VISUALIZATION ---
def visualize_results(processed_data: List[ProcessedSentence], num_samples: int = 3):
    """
    Prints a clean JSON view of the first few sentences.
    """
    # Convert dataclasses to dicts for JSON serialization
    preview_data = [asdict(sent) for sent in processed_data[:num_samples]]
    
    print("\n🔎 VISUAL INSPECTION (JSON PREVIEW):")
    print(json.dumps(preview_data, indent=2))
    print(f"\n... (Total {len(processed_data)} sentences processed)")

In [ ]:
# 1. SETUP: Define your Metadata
book_info = {
    "title": "Brain and Behavior",
    "author": "Eagleman",
    "chapter": "Chapter 1: Neural Foundations"
}

In [ ]:

# 2. INPUT: Your Clean Markdown (Simulated here, replace with your 'clean_text' variable)
markdown_input = destination_file_removedpictures

# 3. RUN THE PIPELINE
processor = NeuroProcessor()
final_sentences = processor.process_text(markdown_input, book_info)

# 4. VISUALIZE
visualize_results(final_sentences, num_samples=3)